### 0-Импорт библиотек

In [1]:
import pandas as pd
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 1-Загрузка и вывод данных

In [2]:
estate_df = pd.read_csv('real_estate_data.csv', index_col='id')
estate_df.head()

C:\Users\leoni\AppData\Local\Temp\ipykernel_13748\1585227361.py:1: DtypeWarning: Columns (9,14) have mixed types. Specify dtype option on import or set low_memory=False.
  estate_df = pd.read_csv('real_estate_data.csv', index_col='id')


,type,sub_type,start_date,end_date,listing_type,tom,building_age,total_floor_count,floor_no,room_count,size,address,furnished,heating_type,price,price_currency
id,,,,,,,,,,,,,,,,
1,Konut,Rezidans,12/10/18,1/9/19,2,30,0,20 ve üzeri,2,2+1,90.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,3500.0,TRY
2,Konut,Daire,2/13/19,NaN,1,14,0,20 ve üzeri,20 ve üzeri,1+0,43.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,490000.0,TRY
3,Konut,Daire,10/9/18,11/8/18,1,30,0,1,Yüksek Giriş,2+1,NaN,Tekirdağ/Çorlu/Reşadiye,NaN,Fancoil,155000.0,TRY
4,Konut,Rezidans,9/10/18,10/10/18,1,30,3,20 ve üzeri,20 ve üzeri,6+1,450.0,İstanbul/Beşiktaş/Levent,NaN,Fancoil,32500000.0,TRY
5,Konut,Rezidans,12/10/18,1/9/19,1,30,0,20 ve üzeri,2,2+1,90.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,1450000.0,TRY


In [3]:
estate_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 403487 entries, 1 to 403487
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   type               403487 non-null  object 
 1   sub_type           403487 non-null  object 
 2   start_date         403487 non-null  object 
 3   end_date           266298 non-null  object 
 4   listing_type       403487 non-null  int64  
 5   tom                403487 non-null  int64  
 6   building_age       376097 non-null  object 
 7   total_floor_count  375466 non-null  object 
 8   floor_no           368191 non-null  object 
 9   room_count         403487 non-null  object 
 10  size               257481 non-null  float64
 11  address            403487 non-null  object 
 12  furnished          0 non-null       float64
 13  heating_type       375517 non-null  object 
 14  price              402772 non-null  float64
 15  price_currency     402772 non-null  object 
dtypes:

In [4]:
def find_cat(data):
    for name in data.columns:
        s: str = ''
        s+=name
        if(data[name].dtype == pd.StringDtype):
            s += ' строка,'
        if(data[name].dtype == pd.Int32Dtype):
            s += ' число,'
        if(data[name].nunique() <=3):
            s += ' мало уникальных'
        print(s)

find_cat(estate_df)

type строка, число, мало уникальных
sub_type строка, число,
start_date строка, число,
end_date строка, число,
listing_type мало уникальных
tom
building_age строка, число,
total_floor_count строка, число,
floor_no строка, число,
room_count строка, число,
size
address строка, число,
furnished мало уникальных
heating_type строка, число,
price
price_currency строка, число,


# 2-Поиск незаполненных значений

In [5]:
estate_df[estate_df.isnull().any(True)]

C:\Users\leoni\AppData\Local\Temp\ipykernel_13748\836468116.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  estate_df[estate_df.isnull().any(True)]


,type,sub_type,start_date,end_date,listing_type,tom,building_age,total_floor_count,floor_no,room_count,size,address,furnished,heating_type,price,price_currency
id,,,,,,,,,,,,,,,,
1,Konut,Rezidans,12/10/18,1/9/19,2,30,0,20 ve üzeri,2,2+1,90.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,3500.0,TRY
2,Konut,Daire,2/13/19,NaN,1,14,0,20 ve üzeri,20 ve üzeri,1+0,43.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,490000.0,TRY
3,Konut,Daire,10/9/18,11/8/18,1,30,0,1,Yüksek Giriş,2+1,NaN,Tekirdağ/Çorlu/Reşadiye,NaN,Fancoil,155000.0,TRY
4,Konut,Rezidans,9/10/18,10/10/18,1,30,3,20 ve üzeri,20 ve üzeri,6+1,450.0,İstanbul/Beşiktaş/Levent,NaN,Fancoil,32500000.0,TRY
5,Konut,Rezidans,12/10/18,1/9/19,1,30,0,20 ve üzeri,2,2+1,90.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,1450000.0,TRY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403483,Konut,Daire,9/18/18,NaN,2,162,NaN,NaN,NaN,+,NaN,İstanbul/Sultanbeyli/Adil,NaN,NaN,1500.0,TRY
403484,Konut,Daire,10/11/18,NaN,1,139,NaN,NaN,NaN,2+1,NaN,Sakarya/Adapazarı/Cumhuriyet,NaN,NaN,120000.0,TRY
403485,Konut,Daire,11/22/18,NaN,1,97,NaN,NaN,NaN,1+1,NaN,Antalya/Alanya/Saray,NaN,NaN,48000.0,EUR


### 2.1-Удаляем пустую колонку

In [6]:
if {'furnished'}.issubset(estate_df.columns):
    estate_df = estate_df.drop('furnished', axis=1)

### 2.2-Подводим дату под одно значение и заменяем пропущенные значения на часто встречаемые

In [7]:
estate_df.start_date = estate_df.start_date.fillna(estate_df.start_date.value_counts().keys()[0])
estate_df.end_date = estate_df.end_date.fillna('12/7/22')

In [8]:
estate_df.start_date = pd.to_datetime(estate_df.start_date, format='%m/%d/%y').dt.strftime('%Y%m%d')

In [9]:
estate_df.end_date = pd.to_datetime(estate_df.end_date, format='%m/%d/%y').dt.strftime('%Y%m%d')

### 2.3-Определение и заполнение нулевых значений

In [10]:
print(estate_df.info())
estate_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 403487 entries, 1 to 403487
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   type               403487 non-null  object 
 1   sub_type           403487 non-null  object 
 2   start_date         403487 non-null  object 
 3   end_date           403487 non-null  object 
 4   listing_type       403487 non-null  int64  
 5   tom                403487 non-null  int64  
 6   building_age       376097 non-null  object 
 7   total_floor_count  375466 non-null  object 
 8   floor_no           368191 non-null  object 
 9   room_count         403487 non-null  object 
 10  size               257481 non-null  float64
 11  address            403487 non-null  object 
 12  heating_type       375517 non-null  object 
 13  price              402772 non-null  float64
 14  price_currency     402772 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 4

,type,sub_type,start_date,end_date,listing_type,tom,building_age,total_floor_count,floor_no,room_count,size,address,heating_type,price,price_currency
id,,,,,,,,,,,,,,,
1,Konut,Rezidans,20181210,20190109,2,30,0,20 ve üzeri,2,2+1,90.0,İstanbul/Kartal/Kordonboyu,Fancoil,3500.0,TRY
2,Konut,Daire,20190213,20221207,1,14,0,20 ve üzeri,20 ve üzeri,1+0,43.0,İstanbul/Kartal/Kordonboyu,Fancoil,490000.0,TRY
3,Konut,Daire,20181009,20181108,1,30,0,1,Yüksek Giriş,2+1,NaN,Tekirdağ/Çorlu/Reşadiye,Fancoil,155000.0,TRY
4,Konut,Rezidans,20180910,20181010,1,30,3,20 ve üzeri,20 ve üzeri,6+1,450.0,İstanbul/Beşiktaş/Levent,Fancoil,32500000.0,TRY
5,Konut,Rezidans,20181210,20190109,1,30,0,20 ve üzeri,2,2+1,90.0,İstanbul/Kartal/Kordonboyu,Fancoil,1450000.0,TRY


In [11]:
categories = ['type', 'sub_type', 'building_age', 'total_floor_count', 'floor_no', 'room_count', 'address', 'heating_type', 'price_currency']
col = ['listing_type', 'tom', 'size', 'price']

for i, cat in enumerate(categories):
    estate_df[cat] =  estate_df[cat].fillna(estate_df[cat].value_counts().keys()[0])

for i, col1 in enumerate(col):
    estate_df[col1] = estate_df[col1].fillna(estate_df[col1].median())

In [12]:
estate_df[estate_df.isnull().any(True)]
estate_df.info()

C:\Users\leoni\AppData\Local\Temp\ipykernel_13748\4065651622.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  estate_df[estate_df.isnull().any(True)]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 403487 entries, 1 to 403487
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   type               403487 non-null  object 
 1   sub_type           403487 non-null  object 
 2   start_date         403487 non-null  object 
 3   end_date           403487 non-null  object 
 4   listing_type       403487 non-null  int64  
 5   tom                403487 non-null  int64  
 6   building_age       403487 non-null  object 
 7   total_floor_count  403487 non-null  object 
 8   floor_no           403487 non-null  object 
 9   room_count         403487 non-null  object 
 10  size               403487 non-null  float64
 11  address            403487 non-null  object 
 12  heating_type       403487 non-null  object 
 13  price              403487 non-null  float64
 14  price_currency     403487 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 4

### 2.4-Предобработка значений для модели МО

In [13]:
#Разделение столбца на три отдельных столбца для лучшей обработки данных с помощью другого датафрейма
new_df = estate_df.address.str.split('/',expand=True)
new_df.columns=['city','district','street']

#Соединение основной и второстепенной датафреймов
estate_df = pd.concat([estate_df,new_df],axis=1)
#Удаление уже ненужного столбца
estate_df = estate_df.drop('address', axis=1)

In [14]:
estate_df = estate_df.replace({'room_count' : {'+':'4+0'}})
#Создание второстепенного датафрейма
new_df = estate_df.room_count.str.split('+',expand=True)
new_df.columns=['room_count1','room_count2']
#преобразование типа этого датафрейма в int
new_df = new_df.astype(int)
#Удаление ненужного столбца
estate_df.drop('room_count', axis=1)
#Внесение изминений в основной датафрейм
estate_df.room_count = new_df.room_count1 + new_df.room_count2

In [15]:
#Предобработка данных столбца building_age
new_df: pd.DataFrame = estate_df.building_age.str.split(' ',expand=True)[0].str.split('-',expand=True)
new_df.columns=['building_age1','building_age2']
#Заполнение пустых значений второстепенного датафрейма
new_df = new_df.fillna(0)
#Преобразование значений в int
new_df = new_df.astype(int)
#Внесение изминений в основной датафрейм
estate_df.building_age = (new_df.building_age1 + new_df.building_age2) / 2

### 2.5-Конечная подготовка, кодирование категориальных значений

In [16]:
# Пометка столбцов которые будут закодированы
estate_df.type = estate_df.type.astype('category')
estate_df.sub_type = estate_df.sub_type.astype('category')
estate_df.total_floor_count = estate_df.total_floor_count.astype('category')
estate_df.floor_no = estate_df.floor_no.astype('category')
estate_df.heating_type = estate_df.heating_type.astype('category')
estate_df.price_currency = estate_df.price_currency.astype('category')
estate_df.city = estate_df.city.astype('category')
estate_df.district = estate_df.district.astype('category')
estate_df.street = estate_df.street.astype('category')
# Создание доп столбцов с закодированными значениями
estate_df.type = estate_df.type.cat.codes
estate_df.sub_type = estate_df.sub_type.cat.codes
estate_df.total_floor_count = estate_df.total_floor_count.cat.codes
estate_df.floor_no = estate_df.floor_no.cat.codes
estate_df.heating_type = estate_df.heating_type.cat.codes
estate_df.price_currency = estate_df.price_currency.cat.codes
estate_df.city = estate_df.city.cat.codes
estate_df.district = estate_df.district.cat.codes
estate_df.street = estate_df.street.cat.codes
estate_df.head(5)

,type,sub_type,start_date,end_date,listing_type,tom,building_age,total_floor_count,floor_no,room_count,size,heating_type,price,price_currency,city,district,street
id,,,,,,,,,,,,,,,,,
1,0,7,20181210,20190109,2,30,0.0,4,20,3,90.0,0,3500.0,2,78,249,2447
2,0,0,20190213,20221207,1,14,0.0,4,21,1,43.0,0,490000.0,2,78,249,2447
3,0,0,20181009,20181108,1,30,0.0,0,41,3,110.0,0,155000.0,2,66,488,3254
4,0,7,20180910,20181010,1,30,1.5,4,21,7,450.0,0,32500000.0,2,78,88,2747
5,0,7,20181210,20190109,1,30,0.0,4,20,3,90.0,0,1450000.0,2,78,249,2447


## 3-Разбиение DataFrame'а на тестовую и тренировочную выборку

In [17]:
y = estate_df.building_age

lab_enc = LabelEncoder()
y = lab_enc.fit_transform(y)
x = estate_df.drop(['building_age', 'listing_type', 'tom', 'size', 'price'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, train_size = 0.8)

### 3.1-Линейная регрессия

In [18]:
linear_regression_model = LogisticRegression(random_state=0).fit(x_train,y_train)
pred = linear_regression_model.predict(x_test)
print(pred)

[0 0 0 ... 0 0 0]


In [19]:
print(accuracy_score(pred, y_test))

0.4140375226151825


### 3.2-Дерево принятия решений

In [20]:
dtm = DecisionTreeClassifier(random_state=0).fit(x_train, y_train)
pred = dtm.predict(x_test)
print(pred)

[ 0  3  7 ... 10  6  4]


In [21]:
print(accuracy_score(pred, y_test))

0.5398894644229101


### 3.3-Алгоритм ближайших соседей

In [22]:
knb = KNeighborsClassifier(n_neighbors=3).fit(x_train, y_train)
pred = knb.predict(x_test)
print(pred)


[ 0  6  7 ... 10  6  0]


In [23]:
print(accuracy_score(pred, y_test))

0.4169620064933456


### 3.4-RandomForestClassifier

In [24]:
rfc = RandomForestClassifier().fit(x_train, y_train)
pred = rfc.predict(x_test)
print(pred)

[ 0  3  7 ... 10  0  4]


In [25]:
print(accuracy_score(pred, y_test))

0.618466380827282
